## Applying unvectorized functions with `apply_ufunc`

This example will illustrate how to conveniently apply an unvectorized function to xarray objects using `apply_ufunc`. This function will expect 1D numpy arrays and return a 1D numpy array. Our goal is to coveniently apply this function over xarray objects that may or may not wrap dask arrays.

First lets load an example dataset

In [4]:
import xarray as xr
import numpy as np

xr.set_options(display_style="html")

air = (xr.tutorial
       .load_dataset("air_temperature")
       .air
       .sortby("lat")  # np.interp needs coordinate in ascending order
       .isel(time=slice(4), lon=slice(3)))  # choose a small subset for convenience
air

<xarray.DataArray 'air' (time: 4, lat: 25, lon: 3)>
array([[[296.29   , 296.79   , 297.1    ],
        [295.9    , 296.19998, 296.79   ],
        [296.6    , 296.19998, 296.4    ],
        [297.     , 296.69998, 296.1    ],
        [295.4    , 295.69998, 295.79   ],
        [293.79   , 294.1    , 294.6    ],
        [293.1    , 293.29   , 293.29   ],
        [290.19998, 290.79   , 291.4    ],
        [287.9    , 288.     , 288.29   ],
        [286.5    , 286.5    , 285.69998],
        [284.6    , 284.9    , 284.19998],
        [282.79   , 283.19998, 282.6    ],
        [280.     , 280.69998, 280.19998],
        [278.4    , 279.     , 279.     ],
        [277.29   , 277.4    , 277.79   ],
        [276.69998, 277.4    , 277.69998],
        [275.9    , 276.9    , 276.9    ],
        [274.79   , 275.19998, 275.6    ],
        [273.69998, 273.6    , 273.79   ],
        [272.1    , 270.9    , 270.     ],
        [274.5    , 274.29   , 274.1    ],
        [266.5    , 267.1    , 267.1    ],
        [250.     , 249.79999, 248.89   ],
        [243.79999, 244.5    , 244.7    ],
        [241.2    , 242.5    , 243.5    ]],

       [[296.29   , 297.19998, 297.4    ],
        [296.19998, 296.69998, 296.79   ],
        [296.4    , 295.9    , 296.19998],
        [295.79   , 295.79   , 296.1    ],
        [295.1    , 295.79   , 295.79   ],
        [294.29   , 294.79   , 294.69998],
        [293.19998, 293.9    , 294.19998],
        [290.19998, 291.1    , 292.4    ],
        [287.5    , 288.5    , 289.4    ],
        [285.69998, 287.4    , 287.1    ],
        [283.29   , 285.19998, 285.19998],
        [281.4    , 282.79   , 283.5    ],
        [279.19998, 280.19998, 280.79   ],
        [277.79   , 278.5    , 279.29   ],
        [276.29   , 276.4    , 277.29   ],
        [275.79   , 276.     , 276.5    ],
        [275.6    , 276.1    , 276.29   ],
        [274.     , 274.4    , 275.1    ],
        [272.1    , 272.69998, 273.19998],
        [269.19998, 268.5    , 267.6    ],
        [272.5    , 271.5    , 270.4    ],
        [269.69998, 269.4    , 268.6    ],
        [253.2    , 252.89   , 252.09999],
        [243.59999, 244.09999, 244.2    ],
        [242.09999, 242.7    , 243.09999]],

       [[296.4    , 296.29   , 296.4    ],
        [296.19998, 296.5    , 296.29   ],
        [295.6    , 295.4    , 295.4    ],
        [295.     , 295.     , 294.9    ],
        [293.9    , 294.79   , 295.     ],
        [293.29   , 293.9    , 294.4    ],
        [292.4    , 292.9    , 293.4    ],
        [289.5    , 289.79   , 291.19998],
        [287.     , 287.1    , 288.29   ],
        [284.9    , 285.6    , 286.4    ],
        [282.     , 283.29   , 284.69998],
        [280.     , 280.79   , 282.4    ],
        [278.6    , 279.1    , 280.     ],
        [277.6    , 277.9    , 278.6    ],
        [276.     , 275.79   , 276.6    ],
        [275.29   , 275.5    , 276.     ],
        [274.5    , 275.6    , 276.1    ],
        [274.29   , 275.     , 275.4    ],
        [275.5    , 275.9    , 275.29   ],
        [273.     , 273.5    , 272.9    ],
        [271.9    , 271.79   , 271.6    ],
        [269.29   , 268.5    , 267.     ],
        [256.19998, 255.5    , 254.2    ],
        [244.59999, 244.39   , 244.     ],
        [242.29999, 242.2    , 242.29999]],

       [[297.5    , 297.69998, 297.5    ],
        [297.     , 297.5    , 297.1    ],
        [296.6    , 296.4    , 296.     ],
        [295.9    , 295.79   , 295.5    ],
        [294.4    , 295.     , 295.4    ],
        [293.     , 293.5    , 294.29   ],
        [291.9    , 291.9    , 292.19998],
        [289.19998, 289.4    , 289.9    ],
        [286.6    , 287.1    , 287.9    ],
        [284.79   , 284.79   , 285.4    ],
        [282.79   , 282.     , 282.69998],
        [281.19998, 280.19998, 280.6    ],
        [279.5    , 278.69998, 278.6    ],
        [278.     , 277.69998, 277.6    ],
        [276.4    , 275.9    , 276.4    ],
        [275.6    , 275.69998, 276.1    ],
        [274.5    , 275.6    

The function we will apply is `np.interp` which expects 1D numpy arrays. This functionality is already implemented in xarray so we use that capability to make sure we are not making mistakes.

In [5]:
newlat = np.linspace(15, 75,100)
%timeit air.interp(lat=newlat)

1.66 ms ± 23.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Let's define a function that works with one vector of data along `lat` at a time.

In [8]:
def interp1d_np(data, x, xi):
    return np.interp(xi, x, data)

interped = interp1d_np(air.isel(time=0, lon=0), air.lat, newlat)
expected = air.interp(lat=newlat)

# no errors are raised if values are equal to within floating point precision
np.testing.assert_allclose(expected.isel(time=0, lon=0).values,
                           interped)

No errors are raised so our interpolation is working.

This function consumes and returns numpy arrays, which means we need to do a lot of work to convert the result back to an xarray object with meaningful metadata. Enter `apply_ufunc` 

    Apply a vectorized function for unlabeled arrays on xarray objects.

    The function will be mapped over the data variable(s) of the input arguments using 
    xarray’s standard rules for labeled computation, including alignment, broadcasting, 
    looping over GroupBy/Dataset variables, and merging of coordinates.
    
`apply_ufunc` has many capabilities but this example will be relatively simple and focus on the relatively common task of vectorizing 1D functions over nD xarray objects. We will iteratively build up the right set of arguments to `apply_ufunc` and read through many error messages in the process of doing so.

In [9]:
xr.apply_ufunc(interp1d_np, # first the function
               air.isel(time=0, lon=0), # now arguments in the order expected by 'interp1_np'
               air.lat,
               newlat)

ValueError: size of dimension 'lat' on inputs was unexpectedly changed by applied function from 25 to 100. Only dimensions specified in ``exclude_dims`` with xarray.apply_ufunc are allowed to change size.

### exclude_dims



`apply_ufunc` needs to know a lot of information about what our function does so that it can reconstruct the outputs. In this case, the size of dimension lat has changed and we need to explicitly specify that this will happen. xarray helpfully tells us that we need to specify the kwarg `exclude_dims`.


```
exclude_dims : set, optional
        Core dimensions on the inputs to exclude from alignment and
        broadcasting entirely. Any input coordinates along these dimensions
        will be dropped. Each excluded dimension must also appear in
        ``input_core_dims`` for at least one argument. Only dimensions listed
        here are allowed to change size between input and output objects.
```

In [10]:
xr.apply_ufunc(interp1d_np, # first the function
               air.isel(time=0, lon=0), # now arguments in the order expected by 'interp1_np'
               air.lat,
               newlat,
               exclude_dims=set(("lat", )),  # dimensions allowed to change size. Must be set!
)

ValueError: each dimension in `exclude_dims` must also be a core dimension in the function signature

### Core dimensions



DESCRIBE CORE DIMENSIONS.

Core dimensions are central to using `apply_ufunc`. In our case, our function expects to receive a 1D vector along `lat` &#x2014; this is the dimension that is "core" to the function's functionality. Multiple core dimensions are possible. `apply_ufunc` needs to know which dimensions of each variable are core dimensions.

    input_core_dims : Sequence[Sequence], optional
        List of the same length as ``args`` giving the list of core dimensions
        on each input argument that should not be broadcast. By default, we
        assume there are no core dimensions on any input arguments.

        For example, ``input_core_dims=[[], ['time']]`` indicates that all
        dimensions on the first argument and all dimensions other than 'time'
        on the second argument should be broadcast.

        Core dimensions are automatically moved to the last axes of input
        variables before applying ``func``, which facilitates using NumPy style
        generalized ufuncs [2]_.
        
    output_core_dims : List[tuple], optional
        List of the same length as the number of output arguments from
        ``func``, giving the list of core dimensions on each output that were
        not broadcast on the inputs. By default, we assume that ``func``
        outputs exactly one array, with axes corresponding to each broadcast
        dimension.

        Core dimensions are assumed to appear as the last dimensions of each
        output in the provided order.
        
Next we specify `"lat"` as `input_core_dims` on both `air` and `air.lat`

In [11]:
xr.apply_ufunc(interp1d_np, # first the function
               air.isel(time=0, lon=0), # now arguments in the order expected by 'interp1_np'
               air.lat,
               newlat,
               input_core_dims=[["lat"], ["lat"], []],
               exclude_dims=set(("lat", )),  # dimensions allowed to change size. Must be set!
)

ValueError: applied function returned data with unexpected number of dimensions: 1 vs 0, for dimensions ()

xarray is telling us that it expected to receive back a numpy array with 0 dimensions but instead received an array with 1 dimension corresponding to `newlat`. We can fix this by specifying `output_core_dims`

In [12]:
xr.apply_ufunc(interp1d_np, # first the function
               air.isel(time=0, lon=0), # now arguments in the order expected by 'interp1_np'
               air.lat,
               newlat,
               input_core_dims=[["lat"], ["lat"], []],  # list with one entry per arg
               output_core_dims=[["lat"]],
               exclude_dims=set(("lat", )),  # dimensions allowed to change size. Must be set!
)

<xarray.DataArray (lat: 100)>
array([296.29000854, 296.19545954, 296.10091053, 296.00636153,
       295.91181252, 296.04848133, 296.21818126, 296.38788119,
       296.55758112, 296.67273227, 296.76970048, 296.8666687 ,
       296.96363692, 296.75757483, 296.36969457, 295.9818143 ,
       295.59393403, 295.20484416, 294.81454468, 294.4242452 ,
       294.03394572, 293.72728105, 293.56000773, 293.39273441,
       293.22546109, 292.92424705, 292.22121083, 291.5181746 ,
       290.81513838, 290.13028509, 289.57271229, 289.01513949,
       288.45756669, 287.8999939 , 287.56060144, 287.22120898,
       286.88181652, 286.54242406, 286.09697099, 285.63636641,
       285.17576183, 284.71515725, 284.27091564, 283.83212835,
       283.39334106, 282.95455378, 282.36727998, 281.69091427,
       281.01454856, 280.33818285, 279.80605987, 279.4181796 ,
       279.03029933, 278.64241906, 278.29908614, 278.02999878,
       277.76091142, 277.49182406, 277.25424934, 277.11121253,
       276.96817571, 276.8251389 , 276.67573964, 276.4818032 ,
       276.28786677, 276.09393033, 275.8999939 , 275.63090654,
       275.36181918, 275.09273182, 274.82364446, 274.55879073,
       274.29454179, 274.03029286, 273.76604392, 273.40907704,
       273.02120417, 272.6333313 , 272.24545843, 272.46364154,
       273.04545824, 273.62727495, 274.20909165, 273.53030303,
       271.59090909, 269.65151515, 267.71212121, 265.        ,
       261.        , 257.        , 253.        , 249.62424168,
       248.12120842, 246.61817516, 245.1151419 , 243.72120019,
       243.09089938, 242.46059857, 241.83029776, 241.19999695])
Coordinates:
    lon      float32 200.0
    time     datetime64[ns] 2013-01-01
Dimensions without coordinates: lat

Finally we get some output! Let's check that this is right



In [13]:
interped = xr.apply_ufunc(interp1d_np, # first the function
               air.isel(time=0, lon=0), # now arguments in the order expected by 'interp1_np'
               air.lat,
               newlat,
               input_core_dims=[["lat"], ["lat"], []],  # list with one entry per arg
               output_core_dims=[["lat"]],
               exclude_dims=set(("lat", )),  # dimensions allowed to change size. Must be set!
)
interped["lat"] = newlat  # need to add this manually
xr.testing.assert_allclose(expected.isel(time=0, lon=0), interped)

It is right!



### Vectorization with `np.vectorize`

Now  our function currently only works on one vector of data which is not so useful given our 3D dataset.
Let's try passing the whole dataset. We add a `print` statement so we can see what our function receives.

In [14]:
def interp1d_np(data, x, xi):
    print(f"data: {data.shape} | x: {x.shape} | xi: {xi.shape}")
    return np.interp(xi, x, data)

interped = xr.apply_ufunc(interp1d_np, # first the function
               air.isel(lon=slice(3), time=slice(4)), # now arguments in the order expected by 'interp1_np'
               air.lat,
               newlat,
               input_core_dims=[["lat"], ["lat"], []],  # list with one entry per arg
               output_core_dims=[["lat"]],
               exclude_dims=set(("lat", )),  # dimensions allowed to change size. Must be set!
)
interped["lat"] = newlat  # need to add this manually
xr.testing.assert_allclose(expected.isel(time=0, lon=0), interped)

data: (4, 3, 25) | x: (25,) | xi: (100,)


ValueError: object too deep for desired array

That's a hard-to-interpret error but our `print` statement output `data: (10, 53, 25) | x: (25,) | xi: (100,)`. `air` has been passed as a 3D numpy array which is not what `np.interp` can handle. Instead we want loop over all combinations of `lon` and `time`; and apply our function to each corresponding vector of data along `lat`.
Turns out this is easy by specifying `vectorize=True` (though not great on performance).

    vectorize : bool, optional
        If True, then assume ``func`` only takes arrays defined over core
        dimensions as input and vectorize it automatically with
        :py:func:`numpy.vectorize`. This option exists for convenience, but is
        almost always slower than supplying a pre-vectorized function.
        Using this option requires NumPy version 1.12 or newer.
        
Also see the documentation for `np.vectorize`: https://docs.scipy.org/doc/numpy/reference/generated/numpy.vectorize.html. Most importantly

    The vectorize function is provided primarily for convenience, not for performance. 
    The implementation is essentially a for loop.

In [15]:
def interp1d_np(data, x, xi):
    print(f"data: {data.shape} | x: {x.shape} | xi: {xi.shape}")
    return np.interp(xi, x, data)

interped = xr.apply_ufunc(interp1d_np, # first the function
               air, # now arguments in the order expected by 'interp1_np'
               air.lat,  # as above
               newlat,  # as above
               input_core_dims=[["lat"], ["lat"], []],  # list with one entry per arg
               output_core_dims=[["lat"]],  # returned data has one dimension
               exclude_dims=set(("lat", )),  # dimensions allowed to change size. Must be set!
               vectorize=True,  # loop over non-core dims
)
interped["lat"] = newlat  # need to add this manually
xr.testing.assert_allclose(expected, interped)

ValueError: shape mismatch: objects cannot be broadcast to a single shape

This unfortunately is another cryptic error from numpy. 

Notice that `newlat` is not an xarray object. Let's add a dimension name `new_lat` and modify the call. Note this cannot be `lat` because xarray expects dimensions to be the same size (or broadcastable) among all inputs. `output_core_dims` needs to be modified appropriately. We'll manually rename `new_lat` back to `lat` for easy checking.

In [16]:
def interp1d_np(data, x, xi):
    print(f"data: {data.shape} | x: {x.shape} | xi: {xi.shape}")
    return np.interp(xi, x, data)

interped = xr.apply_ufunc(interp1d_np, # first the function
               air, # now arguments in the order expected by 'interp1_np'
               air.lat,  # as above
               newlat,  # as above
               input_core_dims=[["lat"], ["lat"], ["new_lat"]],  # list with one entry per arg
               output_core_dims=[["new_lat"]],  # returned data has one dimension
               exclude_dims=set(("lat", )),  # dimensions allowed to change size. Must be a set!
               vectorize=True,  # loop over non-core dims
)
interped = interped.rename({"new_lat": "lat"})
interped["lat"] = newlat  # need to add this manually
xr.testing.assert_allclose(expected.transpose(*interped.dims),  # order of dims is different
                           interped)
interped

data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)


<xarray.DataArray (time: 4, lon: 3, lat: 100)>
array([[[296.29000854, 296.19545954, 296.10091053, ..., 242.46059857,
         241.83029776, 241.19999695],
        [296.79000854, 296.64697173, 296.50393492, ..., 243.46969697,
         242.98484848, 242.5       ],
        [297.1000061 , 297.02485518, 296.94970426, ..., 244.0818167 ,
         243.79090835, 243.5       ]],

       [[296.29000854, 296.26818385, 296.24635916, ..., 242.82726357,
         242.46362721, 242.09999084],
        [297.19998169, 297.07876957, 296.95755745, ..., 243.37878187,
         243.03938941, 242.69999695],
        [297.3999939 , 297.25211866, 297.10424342, ..., 243.63332714,
         243.36665899, 243.09999084]],

       [[296.3999939 , 296.35150609, 296.30301828, ..., 243.41514079,
         242.85756429, 242.29998779],
        [296.29000854, 296.34091556, 296.39182258, ..., 243.26181631,
         242.73090663, 242.19999695],
        [296.3999939 , 296.37333078, 296.34666767, ..., 243.12423614,
         242.71211196, 242.29998779]],

       [[297.5       , 297.37878788, 297.25757576, ..., 244.02817559,
         242.95908749, 241.88999939],
        [297.69998169, 297.65150128, 297.60302087, ..., 243.49695749,
         242.64847264, 241.79998779],
        [297.5       , 297.40303178, 297.30606357, ..., 242.9636286 ,
         242.38180819, 241.79998779]]])
Coordinates:
  * lon      (lon) float32 200.0 202.5 205.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2013-01-01T18:00:00
  * lat      (lat) float64 15.0 15.61 16.21 16.82 ... 73.18 73.79 74.39 75.0

Notice that the printed input shapes are all 1D and correspond to one vector along the `lat` dimension.

The result is now an xarray object with coordinate values copied over from `data`. This is why `apply_ufunc` is so convenient; it takes care of a lot of boilerplate necessary to apply functions that consume and produce numpy arrays to xarray objects.

One final point: `lat` is now the *last* dimension in `interped`. This is a "property" of core dimensions: they are moved to the end before being sent to `interp1d_np` as was noted in the docstring for `input_core_dims`

        Core dimensions are automatically moved to the last axes of input
        variables before applying ``func``, which facilitates using NumPy style
        generalized ufuncs [2]_.

### Parallelization with dask



So far our function can only handle numpy arrays. A real benefit of `apply_ufunc` is the ability to easily parallelize over dask chunks _when needed_. 

We want to apply this function in a vectorized fashion over each chunk of the dask array. This is possible using dask's `blockwise` or `map_blocks`. `apply_ufunc` wraps `blockwise` and asking it to map the function over chunks using `blockwise` is as simple as saying `dask="parallelized"`. With this level of flexibility we need to provide dask with some extra information: 
  1. `output_dtypes`: dtypes of all returned objects, and 
  2. `output_sizes`: lengths of any new dimensions. 
  
Here we need to specify `output_dtypes` since `apply_ufunc` can infer the size of the new dimension `new_lat` from the argument corresponding to the third element in `input_core_dims`. Here I choose the chunk sizes to illustrate that `np.vectorize` is still applied so that our function receives 1D vectors even though the blocks are 3D.

In [17]:
def interp1d_np(data, x, xi):
    print(f"data: {data.shape} | x: {x.shape} | xi: {xi.shape}")
    return np.interp(xi, x, data)

interped = xr.apply_ufunc(interp1d_np, # first the function
               air.chunk({"time": 2, "lon": 2}), # now arguments in the order expected by 'interp1_np'
               air.lat,  # as above
               newlat,  # as above
               input_core_dims=[["lat"], ["lat"], ["new_lat"]],  # list with one entry per arg
               output_core_dims=[["new_lat"]],  # returned data has one dimension
               exclude_dims=set(("lat", )),  # dimensions allowed to change size. Must be a set!
               vectorize=True,  # loop over non-core dims
               dask="parallelized",
               output_dtypes=[float],  # one per output
).rename({"new_lat": "lat"})
interped["lat"] = newlat  # need to add this manually
xr.testing.assert_allclose(expected.transpose(*interped.dims),
                           interped)

data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)data: (25,) | x: (25,) | xi: (100,)

data: (25,) | x: (25,) | xi: (100,)
data: (25,) | x: (25,) | xi: (100,)


Yay! our function is receiving 1D vectors, so we've successfully parallelized applying a 1D function over a block. If you have a distributed dashboard up, you should see computes happening as equality is checked.



### High performance vectorization: gufuncs, numba & guvectorize



`np.vectorize` is a very convenient function but is unfortunately slow. It is only marginally faster than writing a for loop in Python and looping. A common way to get around this is to write a base interpolation function that can handle nD arrays in a compiled language like Fortran and then pass that to `apply_ufunc`.

Another option is to use the numba package which provides a very convenient `guvectorize` decorator: https://numba.pydata.org/numba-doc/latest/user/vectorize.html#the-guvectorize-decorator

Any decorated function gets compiled and will loop over any non-core dimension in parallel when necessary. We need to specify some extra information:
   1. Our function cannot return a variable any more. Instead it must receive a variable (the last argument) whose contents the function will modify. So we change from `def interp1d_np(data, x, xi)` to `def interp1d_np_gufunc(data, x, xi, out)`. Our computed results must be assigned to `out`. All values of `out` must be assigned explicitly.
   
   2. `guvectorize` needs to know the dtypes of the input and output. This is specified in string form as the first argument. Each element of the tuple corresponds to each argument of the function. In this case, we specify `float64` for all inputs and outputs: `"(float64[:], float64[:], float64[:], float64[:])"` corresponding to `data, x, xi, out`
   
   3. Now we need to tell numba the size of the dimensions the function takes as inputs and returns as output i.e. core dimensions. This is done in symbolic form i.e. `data` and `x` are vectors of the same length, say `n`; `xi` and the output `out` have a different length, say `m`. So the second argument is (again as a string)
         `"(n), (n), (m) -> (m)."` corresponding again to `data, x, xi, out`


In [18]:
from numba import float64, guvectorize
@guvectorize("(float64[:], float64[:], float64[:], float64[:])",
             "(n), (n), (m) -> (m)")
def interp1d_np_gufunc(data, x, xi, out):
    # numba doesn't really like this.
    # seem to support fstrings so do it the old way
    print("data: " + str(data.shape) + " | x:" + str(x.shape) + " | xi: " + str(xi.shape))
    out[:] = np.interp(xi, x, data)
    # gufuncs don't return data
    # instead you assign to a the last arg
    # return np.interp(xi, x, data)

<ipython-input-18-89a3ef3b203c>:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "interp1d_np_gufunc" failed type inference due to: Invalid use of Function(<class 'str'>) with argument(s) of type(s): (tuple(int64 x 1))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
In definition 2:
    All templates rejected with literals.
In definition 3:
    All templates rejected without literals.
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<class 'str'>)
[2] During: typing of call at <ipython-input-18-89a3ef3b203c> (7)


File "<ipython-input-18-89a3ef3b203c>", line 7:
def interp1d_np_gufunc(data, x, xi, out):
    <source elided>
    # seem to support fstrings so do it the old way
    print("data: " + str(data.shape) + " | x:" + str(x.shape) + " | xi:

The warnings are about object-mode compilation relating to the `print` statement. This means we don't get much speed up: https://numba.pydata.org/numba-doc/latest/user/performance-tips.html#no-python-mode-vs-object-mode. We'll keep the `print` statement temporarily to make sure that `guvectorize` acts like we want it to.

In [19]:
interped = xr.apply_ufunc(interp1d_np_gufunc, # first the function
               air.chunk({"time": 2, "lon": 2}), # now arguments in the order expected by 'interp1_np'
               air.lat,  # as above
               newlat,  # as above
               input_core_dims=[["lat"], ["lat"], ["new_lat"]],  # list with one entry per arg
               output_core_dims=[["new_lat"]],  # returned data has one dimension
               exclude_dims=set(("lat", )),  # dimensions allowed to change size. Must be a set!
               # vectorize=True,  # not needed since numba takes care of vectorizing
               dask="parallelized",
               output_dtypes=[float],  # one per output
).rename({"new_lat": "lat"})
interped["lat"] = newlat  # need to add this manually
xr.testing.assert_allclose(expected.transpose(*interped.dims),
                           interped)

data: (25,) | x:(25,) | xi: (100,)data: (25,) | x:(25,) | xi: (100,)
data: (25,) | x:(25,) | xi: (100,)
data: (25,) | x:(25,) | xi: (100,)

data: (25,) | x:(25,) | xi: (100,)
data: (25,) | x:(25,) | xi: (100,)data: (25,) | x:(25,) | xi: (100,)
data: (25,) | x:(25,) | xi: (100,)
data: (25,) | x:(25,) | xi: (100,)
data: (25,) | x:(25,) | xi: (100,)
data: (25,) | x:(25,) | xi: (100,)

data: (25,) | x:(25,) | xi: (100,)


Yay! Our function is receiving 1D vectors and is working automatically with dask arrays. Finally let's comment out the print line

In [20]:
from numba import float64, guvectorize
@guvectorize("(float64[:], float64[:], float64[:], float64[:])",
             "(n), (n), (m) -> (m)")
def interp1d_np_gufunc(data, x, xi, out):
    out[:] = np.interp(xi, x, data)

interped = xr.apply_ufunc(interp1d_np_gufunc, # first the function
               air.chunk({"time": 2, "lon": 2}), # now arguments in the order expected by 'interp1_np'
               air.lat,  # as above
               newlat,  # as above
               input_core_dims=[["lat"], ["lat"], ["new_lat"]],  # list with one entry per arg
               output_core_dims=[["new_lat"]],  # returned data has one dimension
               exclude_dims=set(("lat", )),  # dimensions allowed to change size. Must be a set!
               # vectorize=True,  # not needed since numba takes care of vectorizing
               dask="parallelized",
               output_dtypes=[float],  # one per output
).rename({"new_lat": "lat"})
interped["lat"] = newlat  # need to add this manually
xr.testing.assert_allclose(expected.transpose(*interped.dims),
                           interped)

This works at basically the same speed as the original interpolation but the technique is generalizable to any 1D function

In [67]:
%timeit interped.compute()

2.42 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
